# 🐳 Docker pour Data Engineers

Bienvenue dans ce module où tu vas apprendre à **containeriser** tes applications, lancer des services data en quelques secondes, et packager tes pipelines ETL de manière **reproductible et portable** — des compétences indispensables pour un Data Engineer moderne !

---

## Prérequis

| Niveau | Compétence |
|--------|------------|
| ✅ Requis | Avoir suivi les modules `01_intro_data_engineering` et `02_bash_for_data_engineers` |
| ✅ Requis | Connaissances de base en Python |
| ✅ Requis | Avoir accès à un terminal (Linux, Mac, ou Windows avec WSL) |

## Objectifs du module

À la fin de ce module, tu seras capable de :

- Expliquer ce qu'est Docker et pourquoi il est essentiel en Data Engineering
- Installer Docker sur Windows, macOS ou Linux
- Lancer des services data (PostgreSQL, Kafka, Spark…) en une commande
- Écrire un Dockerfile pour packager un script ETL
- Utiliser les volumes, réseaux et docker-compose
- Débugger des containers comme un pro
- Appliquer les bonnes pratiques professionnelles

---

## C'est quoi Docker ?

> **Docker est un outil qui permet d'exécuter des applications dans des environnements isolés, reproductibles et portables**, appelés *containers*.

Au lieu d'installer une application (et toutes ses dépendances) **directement sur ton système**, tu la mets dans une "boîte" (le container) avec tout ce dont elle a besoin.

### Docker ≠ Machine Virtuelle

| Aspect | Machine Virtuelle (VM) | Container Docker |
|--------|----------------------|------------------|
| **Contenu** | OS complet + kernel + drivers + apps | App + libs + système minimal |
| **Taille** | Plusieurs Go | Quelques Mo à centaines de Mo |
| **Démarrage** | Minutes | Secondes/millisecondes |
| **Performance** | Overhead important | Quasi-natif |
| **Isolation** | Complète (hyperviseur) | Au niveau processus |

### Analogies pour bien comprendre

| Analogie | Explication |
|----------|-------------|
| 🧳 **La valise prête** | Un container = une valise déjà remplie. Tu la prends, tu voyages, tu es opérationnel partout. |
| 🍱 **Le tupperware** | Tu prépares un plat, tu le mets dans une boîte hermétique. Chez toi ou ailleurs : c'est le même plat. |
| 📦 **Le zip complet** | Code + librairies + config dans un package. Mais avec la garantie que l'exécution est **identique** partout. |

> ℹ️ **Le savais-tu ?**
>
> Docker a été créé en **2013** par Solomon Hykes chez dotCloud (devenu Docker, Inc.).
> 
> Le nom vient des **dockers** (ouvriers portuaires) qui chargent et déchargent des **containers** sur les bateaux — exactement ce que fait Docker avec les applications !
>
> 📖 [Histoire de Docker sur Wikipedia](https://en.wikipedia.org/wiki/Docker_(software))

---

## 1. Pourquoi Docker est indispensable pour un Data Engineer

En Data Engineering, tu dois souvent :

- Manipuler des **bases de données** (PostgreSQL, MySQL, MongoDB…)
- Lancer des **brokers de messages** (Kafka, RabbitMQ…)
- Déployer des **pipelines ETL**
- Faire tourner des jobs **Spark** ou des APIs

### ❌ Sans Docker

| Problème | Conséquence |
|----------|-------------|
| Installation manuelle complexe | Heures perdues en config |
| Conflits de versions (Java, Python, drivers) | "Ça marchait hier..." |
| Environnements différents (local ≠ prod) | Bugs en production uniquement |
| Onboarding difficile | Nouveaux = 2 jours pour installer |

### ✅ Avec Docker

| Avantage | Exemple concret |
|----------|----------------|
| PostgreSQL en 1 commande | `docker run postgres:16` |
| Test Kafka + Spark sur laptop | Stack complète en local |
| ETL packagé et portable | Tourne identique partout |
| Onboarding en 10 minutes | `docker-compose up` et c'est parti |

> 💡 En résumé : Docker est un **outil central** pour créer des environnements data **reproductibles et industrialisables**. Fini le 😅 *"chez moi ça marche"* !

---

## 2. Concepts clés Docker

Avant d'aller plus loin, maîtrise ces 6 notions fondamentales :

| Concept | Description | Analogie |
|---------|-------------|----------|
| **Image** | Modèle figé (blueprint) contenant OS + dépendances + code | Une *recette* de cuisine |
| **Container** | Instance en cours d'exécution d'une image | Un *plat* préparé à partir de la recette |
| **Registry** | Magasin d'images (Docker Hub, ECR, GHCR) | Un *catalogue de recettes* en ligne |
| **Volume** | Stockage persistant en dehors du container | Un *disque dur externe* branché |
| **Network** | Réseau virtuel entre containers | Un *réseau local* privé |
| **Build context** | Fichiers envoyés à Docker lors du build | Le *dossier de travail* |

### Image Docker

Une image contient :
- Un système de base (ex: `python:3.11-slim`)
- Des bibliothèques (pandas, pyarrow, pyspark…)
- Ton code (scripts, fichiers de config)

**On ne modifie pas une image** — on en reconstruit une nouvelle à partir d'un Dockerfile.

### Container

Un container est une **instance vivante** d'une image :
- Tu crées une image → tu la *lances* → tu obtiens un container
- Tu peux démarrer, arrêter, supprimer un container sans toucher à l'image
- Plusieurs containers peuvent tourner à partir de la même image

### Registry

Un registry stocke et partage les images :
- **Docker Hub** (public/privé) — le plus connu
- **GitHub Container Registry** (GHCR)
- **AWS ECR**, **GCP Artifact Registry**, **Azure ACR**

### Volume

Les données ne doivent pas "mourir" avec le container :
- **Sans volume** : container supprimé = données perdues
- **Avec volume** : données persistantes, partageables

### Network

Permet aux containers de communiquer entre eux :
- Ex: container `etl` se connecte à container `postgres`
- Isolation du réseau de la machine hôte

### Build context

Quand tu fais `docker build -t mon-image .` :
- Le `.` = tout ce que Docker envoie au daemon
- Dossier de 10 Go = envoi de 10 Go 
- D'où l'importance du **`.dockerignore`** !

### Schéma visuel : Architecture Docker

```text
┌─────────────────────────────────────────────────────────────┐
│                      DOCKER HOST                            │
│               (Laptop / Server / Cloud)                     │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  ┌─────────────────────────────────────────────────────┐    │
│  │                  DOCKER ENGINE                      │    │
│  │                                                     │    │
│  │   ┌──────────┐  ┌──────────┐  ┌──────────┐          │    │
│  │   │  IMAGE   │  │  IMAGE   │  │  IMAGE   │          │    │ 
│  │   │ postgres │  │  python  │  │  spark   │          │    │
│  │   └────┬─────┘  └────┬─────┘  └────┬─────┘          │    │
│  │        │             │             │                │    │
│  │        ▼             ▼             ▼                │    │
│  │   ┌──────────┐  ┌──────────┐  ┌──────────┐          │    │
│  │   │CONTAINER │  │CONTAINER │  │CONTAINER │          │    │
│  │   │ de-postgres│ │  de-etl  │  │ de-spark │         │    │
│  │   └────┬─────┘  └──────────┘  └──────────┘          │    │
│  │        │                                            │    │
│  │        ▼                                            │    │
│  │   ┌──────────┐       ┌──────────────┐               │    │
│  │   │  VOLUME  │       │   NETWORK    │               │    │
│  │   │ pg_data  │       │  de-network  │               │    │
│  │   └──────────┘       └──────────────┘               │    │
│  └─────────────────────────────────────────────────────┘    │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

---

## 3. Installation de Docker

| Système | Comment installer |
|---------|------------------|
| 🪟 **Windows** | Docker Desktop + WSL2 |
| 🍎 **macOS** | Docker Desktop (Intel ou Apple Silicon) |
| 🐧 **Linux** | Docker Engine (apt/yum) |

### 🪟 Windows (Docker Desktop + WSL2)

**1. Activer WSL2 :**
```powershell
# Dans PowerShell en administrateur
wsl --install
```

**2. Télécharger Docker Desktop :**
- 🔗 https://www.docker.com/products/docker-desktop/

**3. Installer et redémarrer**

**4. Tester :**
```powershell
docker --version
docker run --rm hello-world
```

---

### 🍎 macOS (Intel & Apple Silicon)

**1. Télécharger Docker Desktop :**
- 🔗 https://www.docker.com/products/docker-desktop/
- Choisir la version **Intel** ou **Apple Silicon (M1/M2/M3)**

**2. Glisser l'app dans Applications**

**3. Lancer Docker Desktop** (icône 🐳 dans la barre)

**4. Tester :**
```bash
docker --version
docker run --rm hello-world
```

---

### 🐧 Linux (Ubuntu/Debian)

```bash
# 1. Mettre à jour et installer les prérequis
sudo apt-get update
sudo apt-get install ca-certificates curl gnupg lsb-release

# 2. Ajouter la clé GPG officielle
sudo mkdir -p /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg

# 3. Ajouter le dépôt Docker
echo "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null

# 4. Installer Docker
sudo apt-get update
sudo apt-get install docker-ce docker-ce-cli containerd.io

# 5. Tester
sudo docker run --rm hello-world

# 6. (Optionnel) Utiliser Docker sans sudo
sudo usermod -aG docker $USER
# Puis déconnexion/reconnexion
```

### Vérifier ton installation

In [ ]:
%%bash
# Vérifier la version de Docker
docker --version

# Vérifier que Docker fonctionne
docker run --rm hello-world

---

## 4. Commandes Docker essentielles

Voici ton **cheat sheet** de base :

### Lancer un container

```bash
docker run image                    # Lancer un container
docker run -d image                 # En arrière-plan (detached)
docker run -it image bash           # Mode interactif
docker run --rm image               # Supprime le container à la fin
docker run --name mon-container image  # Nommer le container
```

### Lister

```bash
docker ps                           # Containers en cours
docker ps -a                        # Tous les containers (y compris stoppés)
docker images                       # Lister les images locales
```

### Stopper / Supprimer

```bash
docker stop CONTAINER_ID            # Arrêter un container
docker rm CONTAINER_ID              # Supprimer un container
docker rmi IMAGE_ID                 # Supprimer une image
```

### Télécharger une image

```bash
docker pull postgres:16             # Télécharger depuis Docker Hub
```

### Logs

```bash
docker logs CONTAINER_ID            # Voir les logs
docker logs -f CONTAINER_ID         # Suivre les logs en temps réel
```

### Raccourcis utiles

| Commande | Description |
|----------|-------------|
| `docker ps -a` | Voir tous les containers |
| `docker logs -f` | Suivre les logs en live |
| `docker exec -it <container> bash` | Entrer dans un container |
| `docker system prune` | Nettoyer les ressources inutilisées |

In [ ]:
%%bash
# Lister les images disponibles localement
echo "=== Images locales ==="
docker images

echo ""
echo "=== Containers en cours ==="
docker ps

echo ""
echo "=== Tous les containers ==="
docker ps -a

---

## 5. Docker pour lancer des services Data

Docker est **extrêmement pratique** pour tester rapidement des services utilisés en Data Engineering.

### PostgreSQL (exemple détaillé)

```bash
docker run -d \
  --name demo-postgres \
  -e POSTGRES_USER=de_user \
  -e POSTGRES_PASSWORD=de_pass \
  -e POSTGRES_DB=de_db \
  -p 5432:5432 \
  postgres:16
```

**Connexion :**
- Host: `localhost`
- Port: `5432`
- User: `de_user`
- Password: `de_pass`
- Database: `de_db`

Tu peux ensuite te connecter avec DBeaver, psql, Python (psycopg2), etc.

---

### Autres services (one-liners)

| Service | Commande |
|---------|----------|
| **Redis** | `docker run -d --name demo-redis -p 6379:6379 redis:latest` |
| **MongoDB** | `docker run -d --name demo-mongo -p 27017:27017 mongo:latest` |
| **Kafka** | `docker run -d --name demo-kafka -p 9092:9092 bitnami/kafka:latest` |
| **Spark** | `docker run -it --name demo-spark bitnami/spark:latest pyspark` |
| **Airflow** | `docker pull apache/airflow:latest` |
| **Jupyter** | `docker run -p 8888:8888 jupyter/scipy-notebook` |

> 💡 **Astuce** : Pour Kafka et Airflow, préfère `docker-compose` car ils nécessitent plusieurs services (Zookeeper, webserver, scheduler…), on le verra un peu plus en bas.

In [ ]:
%%bash
# Lancer PostgreSQL (si Docker est installé)
docker run -d \
  --name demo-postgres \
  -e POSTGRES_USER=de_user \
  -e POSTGRES_PASSWORD=de_pass \
  -e POSTGRES_DB=de_db \
  -p 5432:5432 \
  postgres:16

echo "PostgreSQL lancé sur localhost:5432"

# Vérifier qu'il tourne
docker ps --filter name=demo-postgres

In [ ]:
%%bash
# Nettoyage : stopper et supprimer le container
docker stop demo-postgres 2>/dev/null
docker rm demo-postgres 2>/dev/null
echo "🧹 Container demo-postgres supprimé"

---

## 6. Dockerfile : créer son image

Le **Dockerfile** est un fichier texte qui décrit comment construire une image.

### Instructions principales

| Instruction | Rôle | Exemple |
|-------------|------|--------|
| `FROM` | Image de base | `FROM python:3.11-slim` |
| `WORKDIR` | Répertoire de travail | `WORKDIR /app` |
| `COPY` | Copier des fichiers | `COPY etl.py .` |
| `RUN` | Exécuter une commande (build) | `RUN pip install pandas` |
| `CMD` | Commande par défaut (run) | `CMD ["python", "etl.py"]` |
| `ENTRYPOINT` | Point d'entrée fixe | `ENTRYPOINT ["python"]` |
| `ENV` | Variable d'environnement | `ENV PYTHONUNBUFFERED=1` |
| `EXPOSE` | Port exposé (documentation) | `EXPOSE 8000` |

### Structure de projet recommandée

```text
etl_project/
├── Dockerfile
├── .dockerignore
├── requirements.txt
├── src/
│   ├── etl.py
│   └── utils.py
└── data/               # ⚠️ Ne pas inclure dans l'image !
    └── input.csv
```

> 💡 **Important** : Le Dockerfile doit être à la **racine du service** que tu veux packager.

### Exemple : Dockerfile pour un ETL Python

```dockerfile
# 1. Image de base légère
FROM python:3.11-slim

# 2. Variables d'environnement
ENV PYTHONUNBUFFERED=1
ENV PYTHONDONTWRITEBYTECODE=1

# 3. Répertoire de travail
WORKDIR /app

# 4. Copier et installer les dépendances (cache Docker)
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# 5. Copier le code
COPY src/ ./src/

# 6. Commande par défaut
CMD ["python", "src/etl.py"]
```

### Construire l'image

```bash
cd etl_project
docker build -t etl-image:1.0 .
```

### Lancer le container

```bash
docker run --rm etl-image:1.0
```

---

## 7. Le fichier `.dockerignore`

Le `.dockerignore` empêche d'envoyer des fichiers inutiles dans le build context.

### ❌ Sans `.dockerignore`

- Images **énormes** (datasets inclus)
- Builds **lents** (envoi de Go de données)
- **Fuites de secrets** (.env, clés SSH)

### ✅ Exemple de `.dockerignore` (Data Engineer)

```text
# Données
data/
*.csv
*.parquet
*.json

# Python
__pycache__/
*.pyc
*.pyo
venv/
.venv/
*.egg-info/

# Secrets
.env
*.key
*.pem
secrets/

# Notebooks
*.ipynb
.ipynb_checkpoints/

# Git
.git/
.gitignore

# IDE
.idea/
.vscode/
*.swp

# Logs
logs/
*.log

# OS
.DS_Store
Thumbs.db

# Docker
Dockerfile
docker-compose*.yml
```

> 💡 **Règle d'or** : le `.dockerignore` est **aussi important** que le Dockerfile !

---

## 8. Multi-stage builds

Les **multi-stage builds** permettent de créer des images **plus légères** en séparant :

1. **Stage build** : compilation, installation des dépendances lourdes
2. **Stage runtime** : uniquement ce qui est nécessaire pour exécuter

### Pourquoi c'est utile ?

| Sans multi-stage | Avec multi-stage |
|-----------------|------------------|
| Image de 1.5 Go | Image de 200 Mo |
| Outils de build inclus | Seulement le runtime |
| Surface d'attaque large | Sécurité renforcée |

### Exemple : ETL Python avec multi-stage

```dockerfile
# ============== STAGE 1 : BUILD ==============
FROM python:3.11-slim AS builder

WORKDIR /app

# Installer les dépendances dans un dossier isolé
COPY requirements.txt .
RUN pip install --prefix=/install --no-cache-dir -r requirements.txt

# ============== STAGE 2 : RUNTIME ==============
FROM python:3.11-slim AS runtime

WORKDIR /app

# Copier seulement les dépendances installées
COPY --from=builder /install /usr/local

# Copier le code
COPY src/ ./src/

# Variables d'environnement
ENV PYTHONUNBUFFERED=1

CMD ["python", "src/etl.py"]
```

**Résultat** : image finale **légère** et **sécurisée** ! 🎉

---

## 9. Volumes & Networks

### 9.1 Volumes : persister les données

Les **volumes** permettent de stocker des données en dehors des containers.

**Types de volumes :**

| Type | Syntaxe | Usage |
|------|---------|-------|
| **Bind mount** | `-v /host/path:/container/path` | Données locales (dev) |
| **Volume nommé** | `-v myvolume:/container/path` | Données persistantes (prod) |

**Exemple : monter un dossier local**

```bash
docker run -d \
  --name etl-with-data \
  -v $(pwd)/data:/app/data \
  etl-image:1.0
```

- `$(pwd)/data` → dossier sur ta machine
- `/app/data` → dossier dans le container
- Tu supprimes le container → les données restent dans `./data`

---

### 9.2 Networks : faire communiquer les containers

Par défaut, Docker crée un réseau `bridge`. Tu peux créer un réseau dédié :

```bash
# Créer un réseau
docker network create de-network

# Lancer des containers sur ce réseau
docker run -d --name de-postgres --network de-network postgres:16
docker run -d --name de-etl --network de-network etl-image:1.0
```

**Avantage** : dans le code Python, tu te connectes à `de-postgres` (nom du container) au lieu de `localhost` !

```python
# Dans etl.py
conn = psycopg2.connect(
    host="de-postgres",  # Nom du container !
    database="de_db",
    user="de_user",
    password="de_pass"
)
```

---

## 10. Docker Compose

Quand tu as **plusieurs services** (Postgres + ETL + API…), tu ne veux pas tout lancer à la main.

`docker-compose.yml` permet de **décrire une stack complète** et de la lancer avec **une seule commande**.

### Structure projet avec docker-compose

```text
de-pipeline/
├── docker-compose.yml      # À la racine !
├── etl/
│   ├── Dockerfile
│   ├── requirements.txt
│   └── etl.py
└── data/
    └── input.csv
```

### Exemple : PostgreSQL + ETL

```yaml
version: "3.9"

services:
  postgres:
    image: postgres:16
    container_name: de-postgres
    environment:
      POSTGRES_USER: de_user
      POSTGRES_PASSWORD: de_pass
      POSTGRES_DB: de_db
    ports:
      - "5432:5432"
    volumes:
      - pg_data:/var/lib/postgresql/data
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -U de_user -d de_db"]
      interval: 5s
      timeout: 5s
      retries: 5

  etl:
    build: ./etl
    container_name: de-etl
    depends_on:
      postgres:
        condition: service_healthy
    environment:
      DB_HOST: postgres
      DB_USER: de_user
      DB_PASSWORD: de_pass
      DB_NAME: de_db
    volumes:
      - ./data:/app/data

volumes:
  pg_data:
```

### Commandes docker-compose

```bash
# Lancer la stack
docker compose up -d

# Voir les logs
docker compose logs -f

# Stopper la stack
docker compose down

# Stopper et supprimer les volumes
docker compose down -v

# Reconstruire les images
docker compose up --build
```

---

## 11. Debug Docker

En Data Engineering, tu devras souvent **débugger** un job qui tourne dans un container.

### Voir les logs

```bash
docker logs de-etl                  # Voir les logs
docker logs -f de-etl               # Suivre en temps réel
docker logs --tail 100 de-etl       # Les 100 dernières lignes
```

### Entrer dans un container

```bash
docker exec -it de-etl bash         # Ouvrir un shell bash
docker exec -it de-etl sh           # Pour images Alpine
```

**Cas d'usage :**
- Inspecter les fichiers (`ls`, `cat`)
- Tester une connexion DB (`psql`, `ping`)
- Vérifier les variables d'environnement (`env`)
- Lancer un script manuellement (`python etl.py`)

### Inspecter un container

```bash
docker inspect de-etl               # Détails complets (JSON)
docker inspect --format='{{.NetworkSettings.IPAddress}}' de-etl  # IP du container
```

### Cas Data Engineering typiques

| Problème | Solution |
|----------|----------|
| ETL qui plante sans message | `docker logs de-etl` |
| Connexion DB refusée | `docker exec -it de-etl bash` puis `ping postgres` |
| Spark ne voit pas Kafka | Vérifier les networks Docker |
| Fichier introuvable | `docker exec -it de-etl ls /app/data` |
| Variable d'env manquante | `docker exec -it de-etl env` |

---

## 12. Erreurs fréquentes & Bonnes pratiques

### ❌ Erreurs fréquentes

| Erreur | Conséquence | Solution |
|--------|-------------|----------|
| Pas de `.dockerignore` | Images de plusieurs Go | Créer un `.dockerignore` complet |
| Tout en `latest` | Comportement non reproductible | Tags versionnés (`image:1.0.0`) |
| Dockerfile mal placé | Build context gigantesque | Dockerfile à la racine du service |
| Secrets dans l'image | Fuite de credentials | Variables d'env ou secrets manager |
| Pas de nettoyage | Disque saturé | `docker system prune` régulièrement |
| Données dans l'image | Image énorme, non portable | Utiliser des volumes |
| Versions non fixées | Builds cassés après mise à jour | Fixer les versions dans `requirements.txt` |

### ✅ Bonnes pratiques

| Pratique | Pourquoi |
|----------|----------|
| Images **slim** | `python:3.11-slim` = plus léger et rapide |
| **Multi-stage builds** | Images finales légères |
| **Tags versionnés** | `etl:1.0.0`, `etl:prod`, `etl:staging` |
| **`.dockerignore`** | Builds rapides et sécurisés |
| **Healthchecks** | Savoir si un service est prêt |
| **Volumes pour les données** | Persistance et portabilité |
| **Nettoyage régulier** | `docker system prune` |

### 🧹 Commandes de nettoyage

```bash
# Supprimer les containers arrêtés
docker container prune

# Supprimer les images non utilisées
docker image prune

# Tout nettoyer (⚠️ attention en prod !)
docker system prune -a
```

---

## Quiz de fin de module

Réponds aux questions suivantes pour vérifier tes acquis.

---

### ❓ Q1. Quelle est la différence entre une image et un container Docker ?
a) Une image est un container en cours d'exécution  
b) Un container est une instance en cours d'exécution d'une image  
c) C'est la même chose  
d) Une image contient plusieurs containers

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — Une image est un modèle figé, un container est son exécution vivante.

</details>

---

### ❓ Q2. Pourquoi le fichier `.dockerignore` est-il important ?
a) Pour ignorer les erreurs Docker  
b) Pour réduire la taille du build context et sécuriser les builds  
c) Pour ignorer les logs  
d) C'est optionnel et inutile

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — Il évite d'envoyer des fichiers inutiles (données, secrets) dans le build context.

</details>

---

### ❓ Q3. Quelle commande permet d'entrer dans un container en cours d'exécution ?
a) `docker run -it container bash`  
b) `docker exec -it container bash`  
c) `docker enter container`  
d) `docker ssh container`

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — `docker exec -it <container> bash` ouvre un shell interactif.

</details>

---

### ❓ Q4. À quoi servent les multi-stage builds ?
a) À lancer plusieurs containers en parallèle  
b) À créer des images plus légères en séparant build et runtime  
c) À versionner les images  
d) À gérer les réseaux Docker

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — Multi-stage = image finale légère avec seulement le runtime nécessaire.

</details>

---

### ❓ Q5. Où doit-on placer le fichier `docker-compose.yml` ?
a) Dans le dossier `/etc/docker/`  
b) À la racine du projet multi-services  
c) Dans chaque sous-dossier de service  
d) N'importe où

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — Le `docker-compose.yml` est à la racine du projet, avec les services dans des sous-dossiers.

</details>

---

### ❓ Q6. Quelle commande permet de voir les logs d'un container en temps réel ?
a) `docker logs container`  
b) `docker logs -f container`  
c) `docker watch container`  
d) `docker tail container`

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — L'option `-f` (follow) suit les logs en temps réel.

</details>

---

## Mini-projet : ETL Dockerisé complet

### Objectif
Créer un **pipeline ETL Dockerisé** qui lit un CSV, transforme les données, et les charge dans PostgreSQL.

### Contexte
Tu dois packager un job ETL Python avec Docker et le faire communiquer avec une base PostgreSQL, le tout orchestré par docker-compose.

### Contraintes

1. Créer la structure de projet suivante :
```text
de-mini-projet/
├── docker-compose.yml
├── etl/
│   ├── Dockerfile
│   ├── .dockerignore
│   ├── requirements.txt
│   └── etl.py
└── data/
    └── sales.csv
```

2. L'ETL doit :
   - Lire `data/sales.csv`
   - Calculer une colonne `total = quantity * price`
   - Insérer les données dans PostgreSQL

3. Utiliser un **healthcheck** pour attendre que Postgres soit prêt

4. Les données doivent être montées via un **volume**

### ✅ Solution du mini-projet

<details>
<summary>📥 Afficher la solution complète</summary>

**1. `data/sales.csv`**
```csv
date,product,quantity,price
2024-01-01,Laptop,5,999.99
2024-01-02,Mouse,20,29.99
2024-01-03,Keyboard,15,79.99
2024-01-04,Monitor,8,299.99
2024-01-05,Laptop,3,999.99
```

**2. `etl/requirements.txt`**
```text
pandas==2.1.4
psycopg2-binary==2.9.9
sqlalchemy==2.0.25
```

**3. `etl/.dockerignore`**
```text
__pycache__/
*.pyc
.env
*.log
```

**4. `etl/Dockerfile`**
```dockerfile
FROM python:3.11-slim

ENV PYTHONUNBUFFERED=1

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY etl.py .

CMD ["python", "etl.py"]
```

**5. `etl/etl.py`**
```python
import os
import pandas as pd
from sqlalchemy import create_engine

def main():
    # Configuration depuis variables d'environnement
    db_host = os.environ.get('DB_HOST', 'localhost')
    db_user = os.environ.get('DB_USER', 'de_user')
    db_pass = os.environ.get('DB_PASSWORD', 'de_pass')
    db_name = os.environ.get('DB_NAME', 'de_db')
    
    print("🚀 Démarrage de l'ETL...")
    
    # Extract
    print("📥 Lecture du fichier CSV...")
    df = pd.read_csv('/app/data/sales.csv')
    print(f"   {len(df)} lignes lues")
    
    # Transform
    print("🔄 Transformation des données...")
    df['total'] = df['quantity'] * df['price']
    df['loaded_at'] = pd.Timestamp.now()
    
    # Load
    print("📤 Chargement dans PostgreSQL...")
    engine = create_engine(f'postgresql://{db_user}:{db_pass}@{db_host}/{db_name}')
    df.to_sql('sales', engine, if_exists='replace', index=False)
    
    print("✅ ETL terminé avec succès !")
    print(df.head())

if __name__ == "__main__":
    main()
```

**6. `docker-compose.yml`**
```yaml
version: "3.9"

services:
  postgres:
    image: postgres:16
    container_name: de-postgres
    environment:
      POSTGRES_USER: de_user
      POSTGRES_PASSWORD: de_pass
      POSTGRES_DB: de_db
    ports:
      - "5432:5432"
    volumes:
      - pg_data:/var/lib/postgresql/data
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -U de_user -d de_db"]
      interval: 5s
      timeout: 5s
      retries: 5

  etl:
    build: ./etl
    container_name: de-etl
    depends_on:
      postgres:
        condition: service_healthy
    environment:
      DB_HOST: postgres
      DB_USER: de_user
      DB_PASSWORD: de_pass
      DB_NAME: de_db
    volumes:
      - ./data:/app/data

volumes:
  pg_data:
```

**7. Lancer le projet :**
```bash
cd de-mini-projet
docker compose up --build
```

**8. Vérifier les données dans Postgres :**
```bash
docker exec -it de-postgres psql -U de_user -d de_db -c "SELECT * FROM sales;"
```

</details>

---

## 📚 Ressources pour aller plus loin

### 🌐 Documentation officielle
- [Docker Docs](https://docs.docker.com/) — Documentation officielle
- [Docker Hub](https://hub.docker.com/) — Registry d'images publiques
- [Dockerfile reference](https://docs.docker.com/engine/reference/builder/) — Toutes les instructions

### 🎮 Pratique
- [Play with Docker](https://labs.play-with-docker.com/) — Environnement Docker gratuit en ligne
- [Docker 101 Tutorial](https://www.docker.com/101-tutorial/) — Tutoriel officiel interactif

### 📦 Images utiles pour Data Engineering
- [postgres](https://hub.docker.com/_/postgres) — Base de données relationnelle
- [apache/airflow](https://hub.docker.com/r/apache/airflow) — Orchestrateur
- [bitnami/spark](https://hub.docker.com/r/bitnami/spark) — Traitement distribué
- [bitnami/kafka](https://hub.docker.com/r/bitnami/kafka) — Streaming
- [jupyter/scipy-notebook](https://hub.docker.com/r/jupyter/scipy-notebook) — Notebooks

---

## ➡️ Prochaine étape

Maintenant que tu maîtrises Docker, passons à l'orchestration de containers **à grande échelle** !

👉 **Module suivant : `15_kubernetes_fundamentals`** — Orchestrer des containers avec Kubernetes

Tu vas apprendre :
- Pods, Deployments, Services
- ConfigMaps, Secrets
- Spark et Airflow sur Kubernetes

---

🎉 **Félicitations !** Tu as terminé le module Docker pour Data Engineers.